[Retrieve IRAF observatories](http://tdc-www.harvard.edu/iraf/rvsao/bcvcorr/obsdb.html)

In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import numpy as np
from astropy.time import Time
from astropy.coordinates import (SkyCoord, EarthLocation, Latitude, Longitude)
import astropy.units as u
from astropy.utils import data
from astropy.io import ascii


In [2]:
# Grabbed from: http://tdc-www.harvard.edu/iraf/rvsao/bcvcorr/obsdb.html
# added altitude=0m to Observatorio Astronomico Nacional, Tonantzintla
# corrected Whipple observatory latitude from "31.41.3" to "31:41.3"

IRAF = '''
observatory = "kpno"
    name = "Kitt Peak National Observatory"
    longitude = 111:36.0
    latitude = 31:57.8
    altitude = 2120.
    timezone = 7

observatory = "ctio"
    name = "Cerro Tololo Interamerican Observatory"
    longitude = 70.815
    latitude = -30.16527778
    altitude = 2215.
    timezone = 4

observatory = "eso"
    name = "European Southern Observatory"
    longitude = 70:43.8
    latitude = -29:15.4
    altitude = 2347
    timezone = 4

observatory = "lick"
    name = "Lick Observatory"
    longitude = 121:38.2
    latitude = 37:20.6
    altitude = 1290
    timezone = 8

observatory = "mmto"
    name = "Multiple Mirror Telescope Observatory"
    longitude = 110:53.1
    latitude = 31:41.3
    altitude = 2600
    timezone = 7

observatory = "mmt"
    name = "Whipple Observatory"
    longitude = 110:53.1
    latitude = 31:41.3
    altitude = 2608
    timezone = 7

observatory = "flwo"
    name = "Whipple Observatory"
    longitude = 110:52:39
    latitude = 31:40:51.4
    altitude = 2320
    timezone = 7

observatory = "cfht"
    name = "Canada-France-Hawaii Telescope"
    longitude = 155:28.3
    latitude = 19:49.6
    altitude = 4215
    timezone = 10

observatory = "lapalma"
    name = "Roque de los Muchachos, La Palma"
    longitude = 17:52.8
    latitude = 28:45.5
    altitude = 2327
    timezone = 0

observatory = "mso"
    name = "Mt. Stromlo Observatory"
    longitude = 210:58:32.4
    latitude = -35:19:14.34
    altitude = 767
    timezone = -10

observatory = "sso"
    name = "Siding Spring Observatory"
    longitude = 210:56:19.70
    latitude = -31:16:24.10
    altitude = 1149
    timezone = -10

observatory = "aao"
    name = "Anglo-Australian Observatory"
    longitude = 210:56:2.09
    latitude = -31:16:37.34
    altitude = 1164
    timezone = -10

observatory = "mcdonald"
    name = "McDonald Observatory"
    longitude = 104.0216667
    latitude = 30.6716667
    altitude = 2075
    timezone = 6

observatory = "lco"
    name = "Las Campanas Observatory"
    longitude = 70:42.1
    latitude = -29:0.2
    altitude = 2282
    timezone = 4

# Submitted by Alan Koski 1/13/93
observatory = "mtbigelow"
    name = "Catalina Observatory: 61 inch telescope"
    longitude = 110:43.9
    latitude = 32:25.0
    altitude = 2510.
    timezone = 7

# Revised by Daniel Durand 2/23/93
observatory = "dao"
    name = "Dominion Astrophysical Observatory"
    longitude = 123:25.0
    latitude = 48:31.3
    altitude = 229
    timezone = 8

# Submitted by Patrick Vielle 5/4/93
observatory = "spm"
    name = "Observatorio Astronomico Nacional, San Pedro Martir"
    longitude = 115:29:13
    latitude = 31:01:45
    altitude = 2830
    timezone = 7

# Submitted by Patrick Vielle 5/4/93
observatory = "tona"
    name = "Observatorio Astronomico Nacional, Tonantzintla"
    longitude = 98:18:50
    latitude = 19:01:58
    altitude = 0
    timezone = 8

# Submitted by Don Hamilton 8/18/93
observatory = "Palomar"
    name = "The Hale Telescope"
    longitude = 116:51:46.80
    latitude = 33:21:21.6
    altitude = 1706.
    timezone = 8

# Submitted by Pat Seitzer 10/31/93
observatory = "mdm"
    name = "Michigan-Dartmouth-MIT Observatory"
    longitude = 111:37.0
    latitude = 31:57.0
    altitude = 1938.5
    timezone = 7

# Submitted by Ignacio Ferrin 9/1/94
observatory = "NOV"
    name = "National Observatory of Venezuela"
    longitude = 70:52.0
    latitude = 8:47.4
    altitude = 3610
    timezone = 4

# Submitted by Alan Welty 10/28/94
observatory = "bmo"
    name = "Black Moshannon Observatory"
    longitude = 78:00.3
    latitude = 40:55.3
    altitude = 738
    timezone = 5

# Submitted by Biwei JIANG 11/28/95
observatory = "BAO"
    name = "Beijing XingLong Observatory"
    longitude = 242:25.5
    latitude = 40:23.6
    altitude = 950.
    timezone = -8

# From Astronomical Almanac 1996
observatory = "keck"
    name = "W. M. Keck Observatory"
    longitude = 155:28.7
    latitude = 19:49.7
    altitude = 4160
    timezone = 10

# Padova Astronomical Observatory, Asiago, Italy.
# Submitted by Lina Tomasella 6/11/96
observatory = "ekar"
    name = "Mt. Ekar 182 cm. Telescope"
    longitude = 348:25:07.92
    latitude = 45:50:54.92
    altitude = 1413.69
    timezone = -1

# Submitted by Michael Ledlow 8/8/96
observatory = "apo"
    name = "Apache Point Observatory"
    longitude = 105:49.2
    latitude = 32:46.8
    altitude = 2798.
    timezone = 7 

# Submitted by Michael Ledlow 8/8/96
observatory = "lowell"
    name = "Lowell Observatory"
    longitude = 111:32.1
    latitude = 35:05.8
    altitude = 2198. 
    timezone = 7 

# Submitted by S.G. Bhargavi 8/12/96
observatory = "vbo"
    name = "Vainu Bappu Observatory"
    longitude = 281.1734
    latitude = 12.57666
    altitude = 725. 
    timezone = -5.5
'''

In [3]:
codes = []
names = []
longitudes = []
latitudes = []
altitudes = []
timezones = []

for line in IRAF.splitlines():
    if '=' in line:
        strip_line = line.strip()
        value = line.split('=')[1].strip()

        if strip_line.startswith('observatory'):
            codes.append(value[1:-1])
        elif strip_line.startswith('name'):
            names.append(value[1:-1])
        elif strip_line.startswith('longitude'):
            longitudes.append(Longitude(value, unit=u.deg))
        elif strip_line.startswith('latitude'):
            latitudes.append(Latitude(value, unit=u.deg))
        elif strip_line.startswith('altitude'):
            altitudes.append(float(value))
        elif strip_line.startswith('timezone'):
            timezones.append(float(value))

from astropy.table import Table
longitudes = Longitude(longitudes)
latitudes = Latitude(latitudes)
altitudes = np.array(altitudes)*u.m

tbl = Table([codes, names, longitudes, latitudes, altitudes], 
            names=('Code', 'Name', 'Longitude', 'Latitude', 'Altitude'), 
            dtype=(str, str, float, float, float))

In [4]:
def from_code(code):
    i = np.argwhere(tbl['Code'] == code)[0][0]
    return EarthLocation.from_geodetic(tbl['Longitude'][i], 
                                       tbl['Latitude'][i], 
                                       tbl['Altitude'][i])

def from_name(code):
    i = np.argwhere(tbl['Name'] == code)[0][0]
    return EarthLocation.from_geodetic(tbl['Longitude'][i], 
                                       tbl['Latitude'][i], 
                                       tbl['Altitude'][i])

print(from_code('keck').to_geodetic())
print(from_name('W. M. Keck Observatory').to_geodetic())

(<Longitude 155.47833333333332 deg>, <Latitude 19.828333333333326 deg>, <Quantity 4160.000000000752 m>)
(<Longitude 155.47833333333332 deg>, <Latitude 19.828333333333326 deg>, <Quantity 4160.000000000752 m>)
